In [1]:
# -*- coding: utf-8 -*-

from matplotlib import pyplot as plt
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch
import os
from torch.optim import *
from torch import nn
from torch.nn import functional as F
from scipy import ndimage
import pandas as pd
from copy import deepcopy
import random
from sklearn.metrics import f1_score

random.seed(7)
torch.cuda.manual_seed(7)
torch.manual_seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [2]:
DATA_ROOT = '/beegfs/bva212/openmic-2018'
OPENMIC = np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz'))
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

In [3]:
len_data = Y_mask.shape[0]
idx_train = np.random.choice(len_data, int(len_data*0.7), replace=False)
remain_set = set(np.arange(len_data))-set(idx_train)
idx_test = np.random.choice(list(remain_set), int(len_data*0.1), replace=False)
idx_val = list(remain_set-set(idx_test))

In [4]:
Y_mask_val = Y_mask[idx_val]
Y_mask_test = Y_mask[idx_test]

label_val = Y_true[idx_val]
label_test = Y_true[idx_test]

In [5]:
class ArrowOfTime(Dataset):

    def __init__(self, root_dir, files, weights, label):
        self.weights = weights
        self.device = device
        self.root_dir = root_dir
        self.files = files
        self.label = label

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        logscalogram = (np.load(self.root_dir + self.files[idx]+'_cqt.npy') - -24.3633)/14.2659
        len_cqt = logscalogram.shape[1]
        rem = len_cqt % 3
        logscalograms = np.split(logscalogram[:, :-rem], 3, axis = 1)
        logscalograms = np.stack(logscalograms)

        weight = self.weights[idx]
        label = (self.label[idx] >0.5).astype(int)
        return {'logscalogram': logscalograms, 'label': label[np.newaxis, :], 'weight': weight[np.newaxis, :]}

filenames = []
root_dir = '/beegfs/bva212/openmic-2018/cqt_full/'

BATCH_SIZE = 32

def my_collate(batch):
    data = np.stack([item['logscalogram'] for item in batch])
    target = np.concatenate([item['label'] for item in batch],axis=0)
    weight = np.concatenate([item['weight'] for item in batch],axis=0)
    weight_sum = weight.sum(0)
    weight_sum = np.repeat(weight_sum[np.newaxis, :], len(batch), 0)
    weight_sum[weight_sum==0]=1
    weight = weight/weight_sum
    return [torch.from_numpy(data).float(), torch.from_numpy(target).float(), torch.from_numpy(weight).float()]


In [6]:
class snet2_jigsaw(nn.Module):

    def __init__(self, output_classes = 20):
        '''
        Create the 5 Conv Layer Sound Net network architecture as per the paper - https://arxiv.org/pdf/1610.09001.pdf
        '''
        super(snet2_jigsaw, self).__init__()

        self.conv_layers = nn.Sequential(nn.Conv2d(in_channels = 1, out_channels= 16, kernel_size = 5, stride = 2, padding = 5), 
                                nn.BatchNorm2d(num_features = 16), 
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 2, padding = 5),
                                nn.BatchNorm2d(32),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 5),
                                nn.BatchNorm2d(64),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(128),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(256),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(512),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(1024),
                                nn.ReLU(inplace = True),
                                nn.AdaptiveAvgPool2d(output_size = 1)
                                )
        self.concat_mlp_layer = nn.Sequential(nn.Linear(3072, 2048),
                                              nn.BatchNorm1d(num_features = 2048), 
                                              nn.ReLU(inplace = True),
                                              
                                              nn.Linear(2048, 1024),
                                              nn.BatchNorm1d(num_features = 1024), 
                                              nn.ReLU(inplace = True),
                                              
                                              nn.Linear(1024, 256),
                                              nn.BatchNorm1d(num_features = 256), 
                                              nn.ReLU(inplace = True),
                                             )
        self.mlp_layer = nn.Linear(256, output_classes)
              
    def forward(self, input):
        conv_strips = []
        n_strips = input.shape[1]
        for strip in range(n_strips):
            conv_strip = input[:,strip]
            conv_strip = conv_strip.unsqueeze(1)
            conv_strips.append(self.conv_layers(conv_strip))

        concat_out=torch.cat(conv_strips,1)
        out = self.concat_mlp_layer(concat_out.view(concat_out.shape[0], -1))
        output = self.mlp_layer(out.view(out.shape[0], -1))
        return output
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
        
# Function for testing the model
def test_model(loader, model):
    correct = 0
    total_loss = 0
    total = 0
    total_num = 0
    actual_arr = []
    predicted_arr = []
    weight_array = []
    model.eval()
    with torch.no_grad():
        for spectrogram, target, weight in loader:
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            predicted = (torch.sigmoid(outputs.data)>0.5).float()
            
            actual_arr.extend(target.view(1,-1).squeeze().numpy().astype(int).tolist())
            predicted_arr.extend(predicted.view(1,-1).squeeze().cpu().numpy().astype(int).tolist())
            
            total_loss += loss.item()
            total += weight_batch.shape[0]
            
            correct += ((weight_batch != 0).float()*(predicted.eq(target_batch.view_as(predicted)).float())).sum().item()
            total_num += (weight_batch != 0).sum().item()
            weight_array = np.concatenate((weight_array,(weight != 0).reshape(-1).numpy().astype(int)))
        accuracy = (100 * correct / total_num)
        return accuracy, f1_score(actual_arr, predicted_arr, average='micro', sample_weight = weight_array), total_loss/total

def train_model(train_loader, val_loader, model, optimizer, scheduler, num_epochs):
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    train_f1_list = []
    val_f1_list = []
    best_val_acc = 0
    for epoch in range(num_epochs):
        for spectrogram, target, weight in train_loader:
            model.train()
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            optimizer.zero_grad()
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            loss.backward()
            optimizer.step()
        train_acc, f1_score_train, train_loss = test_model(train_loader, model)
        val_acc, f1_score_val, val_loss = test_model(val_loader, model)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state_dict = deepcopy(model.state_dict())
        
        train_acc_list.append(train_acc)
        train_f1_list.append(f1_score_train)
        train_loss_list.append(train_loss)
        val_f1_list.append(f1_score_val)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
        
        scheduler.step(val_acc)
        print("Epoch:{}".format(epoch+1))
        print("Validation Accuracy:{:.2f}, Validation F1:{:.2f}, Val Loss: {:.5f}".format(val_acc, f1_score_val, val_loss))
        print("Training Acc: {:.2f}, Training F1:{:.2f}, Train Loss: {:.5f}".format(train_acc, f1_score_train, train_loss))
    return train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict

In [7]:
import pickle
sizes = [10, 50, 250, 500, 1000, 1500]

Model_Data = pickle.load(open('List_indices_data_size_exp.pkl', 'rb'))

Val_dataset = ArrowOfTime(root_dir, sample_key[idx_val], Y_mask_val, label_val)
Val_loader = torch.utils.data.DataLoader(dataset = Val_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

Test_dataset = ArrowOfTime(root_dir, sample_key[idx_test], Y_mask_test, label_test)
Test_loader = torch.utils.data.DataLoader(dataset = Test_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

results_dict = {}

for i in range(len(sizes)):
    if sizes[i] != 1500:
        idx_train_new = Model_Data[i]
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[idx_train]
        label_train = Y_true[idx_train]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    weights = torch.load('/beegfs/sc6957/capstone/models/20191106/snet2_jigsaw_large_best_model.pth')['modelStateDict']
    model = snet2_jigsaw(2)
    model.load_state_dict(weights)
    model.mlp_layer = nn.Linear(256, 20)
    model.to(device)
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('Size_Exp_Jigsaw_Wt_Results.pkl'.format(sizes[i]),'wb'))

Epoch:1
Validation Accuracy:55.11, Validation F1:0.55, Val Loss: 0.41050
Training Acc: 54.05, Training F1:0.54, Train Loss: 0.43966
Epoch:2
Validation Accuracy:54.58, Validation F1:0.55, Val Loss: 0.41253
Training Acc: 54.95, Training F1:0.55, Train Loss: 0.44266
Epoch:3
Validation Accuracy:53.13, Validation F1:0.53, Val Loss: 0.41997
Training Acc: 57.43, Training F1:0.57, Train Loss: 0.41728
Epoch     3: reducing learning rate of group 0 to 1.0000e-06.
Epoch:4
Validation Accuracy:53.00, Validation F1:0.53, Val Loss: 0.41742
Training Acc: 60.36, Training F1:0.60, Train Loss: 0.42180
Epoch:5
Validation Accuracy:53.72, Validation F1:0.54, Val Loss: 0.41718
Training Acc: 61.49, Training F1:0.61, Train Loss: 0.42038
Epoch:6
Validation Accuracy:53.59, Validation F1:0.54, Val Loss: 0.41448
Training Acc: 60.81, Training F1:0.61, Train Loss: 0.39866
Epoch     6: reducing learning rate of group 0 to 1.0000e-07.
Epoch:7
Validation Accuracy:54.00, Validation F1:0.54, Val Loss: 0.41987
Training Ac

Epoch:12
Validation Accuracy:59.48, Validation F1:0.59, Val Loss: 0.39850
Training Acc: 79.42, Training F1:0.79, Train Loss: 0.34019
Epoch:13
Validation Accuracy:58.22, Validation F1:0.58, Val Loss: 0.40095
Training Acc: 80.87, Training F1:0.81, Train Loss: 0.32854
Epoch:14
Validation Accuracy:60.08, Validation F1:0.60, Val Loss: 0.39309
Training Acc: 83.91, Training F1:0.84, Train Loss: 0.31843
Epoch:15
Validation Accuracy:60.53, Validation F1:0.61, Val Loss: 0.39526
Training Acc: 84.64, Training F1:0.85, Train Loss: 0.31141
Epoch:16
Validation Accuracy:60.86, Validation F1:0.61, Val Loss: 0.39511
Training Acc: 85.85, Training F1:0.86, Train Loss: 0.30734
Epoch:17
Validation Accuracy:61.53, Validation F1:0.62, Val Loss: 0.39152
Training Acc: 84.11, Training F1:0.84, Train Loss: 0.30515
Epoch:18
Validation Accuracy:62.63, Validation F1:0.63, Val Loss: 0.38665
Training Acc: 86.91, Training F1:0.87, Train Loss: 0.30261
Epoch:19
Validation Accuracy:61.94, Validation F1:0.62, Val Loss: 0.3

Epoch:23
Validation Accuracy:75.93, Validation F1:0.76, Val Loss: 0.29985
Training Acc: 95.80, Training F1:0.96, Train Loss: 0.15848
Epoch    23: reducing learning rate of group 0 to 1.0000e-06.
Epoch:24
Validation Accuracy:76.03, Validation F1:0.76, Val Loss: 0.29671
Training Acc: 96.37, Training F1:0.96, Train Loss: 0.14969
Epoch:25
Validation Accuracy:76.11, Validation F1:0.76, Val Loss: 0.29859
Training Acc: 97.23, Training F1:0.97, Train Loss: 0.14153
Epoch:26
Validation Accuracy:76.16, Validation F1:0.76, Val Loss: 0.29788
Training Acc: 96.88, Training F1:0.97, Train Loss: 0.14030
Epoch:27
Validation Accuracy:76.05, Validation F1:0.76, Val Loss: 0.29828
Training Acc: 97.17, Training F1:0.97, Train Loss: 0.14028
Epoch:28
Validation Accuracy:76.05, Validation F1:0.76, Val Loss: 0.29668
Training Acc: 97.27, Training F1:0.97, Train Loss: 0.14244
Epoch    28: reducing learning rate of group 0 to 1.0000e-07.
Epoch:29
Validation Accuracy:75.80, Validation F1:0.76, Val Loss: 0.30086
Trai

Epoch:32
Validation Accuracy:80.72, Validation F1:0.81, Val Loss: 0.25764
Training Acc: 96.05, Training F1:0.96, Train Loss: 0.12263
Epoch:33
Validation Accuracy:81.16, Validation F1:0.81, Val Loss: 0.25471
Training Acc: 96.85, Training F1:0.97, Train Loss: 0.11822
Epoch:34
Validation Accuracy:81.18, Validation F1:0.81, Val Loss: 0.25237
Training Acc: 96.97, Training F1:0.97, Train Loss: 0.11749
Epoch:35
Validation Accuracy:81.30, Validation F1:0.81, Val Loss: 0.25296
Training Acc: 97.00, Training F1:0.97, Train Loss: 0.11516
Epoch:36
Validation Accuracy:80.62, Validation F1:0.81, Val Loss: 0.26065
Training Acc: 95.76, Training F1:0.96, Train Loss: 0.12223
Epoch:37
Validation Accuracy:81.31, Validation F1:0.81, Val Loss: 0.25355
Training Acc: 96.94, Training F1:0.97, Train Loss: 0.12001
Epoch:38
Validation Accuracy:80.75, Validation F1:0.81, Val Loss: 0.25743
Training Acc: 95.83, Training F1:0.96, Train Loss: 0.12347
Epoch:39
Validation Accuracy:81.00, Validation F1:0.81, Val Loss: 0.2

Epoch:43
Validation Accuracy:89.20, Validation F1:0.89, Val Loss: 0.16089
Training Acc: 99.79, Training F1:1.00, Train Loss: 0.03031
Epoch:44
Validation Accuracy:89.27, Validation F1:0.89, Val Loss: 0.16668
Training Acc: 99.83, Training F1:1.00, Train Loss: 0.02856
Epoch:45
Validation Accuracy:89.14, Validation F1:0.89, Val Loss: 0.16893
Training Acc: 99.79, Training F1:1.00, Train Loss: 0.03032
Epoch:46
Validation Accuracy:89.12, Validation F1:0.89, Val Loss: 0.16475
Training Acc: 99.83, Training F1:1.00, Train Loss: 0.02812
Epoch:47
Validation Accuracy:89.26, Validation F1:0.89, Val Loss: 0.16445
Training Acc: 99.78, Training F1:1.00, Train Loss: 0.02921
Epoch:48
Validation Accuracy:89.03, Validation F1:0.89, Val Loss: 0.15925
Training Acc: 99.76, Training F1:1.00, Train Loss: 0.02979
Epoch:49
Validation Accuracy:89.15, Validation F1:0.89, Val Loss: 0.16941
Training Acc: 99.76, Training F1:1.00, Train Loss: 0.03011
Epoch:50
Validation Accuracy:89.06, Validation F1:0.89, Val Loss: 0.1

In [8]:
import pickle
sizes = [10, 50, 250, 500, 1000, 1500]

Model_Data = pickle.load(open('List_indices_data_size_exp.pkl', 'rb'))

Val_dataset = ArrowOfTime(root_dir, sample_key[idx_val], Y_mask_val, label_val)
Val_loader = torch.utils.data.DataLoader(dataset = Val_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

Test_dataset = ArrowOfTime(root_dir, sample_key[idx_test], Y_mask_test, label_test)
Test_loader = torch.utils.data.DataLoader(dataset = Test_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

results_dict = {}
model1 = snet2_jigsaw(20).to(device)
model = snet2_jigsaw(20).to(device)

for i in range(len(sizes)):
    if sizes[i] != 1500:
        idx_train_new = Model_Data[i]
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[idx_train]
        label_train = Y_true[idx_train]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    model.load_state_dict(model1.state_dict())
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
    pickle.dump(results_dict, open('Size_Exp_Random_Wt_Results_size_{}.pkl'.format(sizes[i]),'wb'))

Epoch:1
Validation Accuracy:44.34, Validation F1:0.44, Val Loss: 0.42147
Training Acc: 47.07, Training F1:0.47, Train Loss: 0.46536
Epoch:2
Validation Accuracy:44.40, Validation F1:0.44, Val Loss: 0.42465
Training Acc: 48.20, Training F1:0.48, Train Loss: 0.43732
Epoch:3
Validation Accuracy:48.37, Validation F1:0.48, Val Loss: 0.42175
Training Acc: 56.98, Training F1:0.57, Train Loss: 0.42881
Epoch:4
Validation Accuracy:51.21, Validation F1:0.51, Val Loss: 0.42034
Training Acc: 66.44, Training F1:0.66, Train Loss: 0.39865
Epoch:5
Validation Accuracy:53.99, Validation F1:0.54, Val Loss: 0.41664
Training Acc: 77.48, Training F1:0.77, Train Loss: 0.40210
Epoch:6
Validation Accuracy:54.51, Validation F1:0.55, Val Loss: 0.41425
Training Acc: 85.36, Training F1:0.85, Train Loss: 0.38271
Epoch:7
Validation Accuracy:55.70, Validation F1:0.56, Val Loss: 0.40974
Training Acc: 97.07, Training F1:0.97, Train Loss: 0.30790
Epoch:8
Validation Accuracy:56.44, Validation F1:0.56, Val Loss: 0.40560
Tra

Epoch:12
Validation Accuracy:65.64, Validation F1:0.66, Val Loss: 0.36306
Training Acc: 99.90, Training F1:1.00, Train Loss: 0.13589
Epoch:13
Validation Accuracy:64.95, Validation F1:0.65, Val Loss: 0.37041
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.12867
Epoch:14
Validation Accuracy:66.75, Validation F1:0.67, Val Loss: 0.35992
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.11632
Epoch:15
Validation Accuracy:65.74, Validation F1:0.66, Val Loss: 0.36394
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.11196
Epoch:16
Validation Accuracy:66.45, Validation F1:0.66, Val Loss: 0.35836
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.09771
Epoch    16: reducing learning rate of group 0 to 1.0000e-06.
Epoch:17
Validation Accuracy:66.55, Validation F1:0.67, Val Loss: 0.36008
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.09772
Epoch:18
Validation Accuracy:66.99, Validation F1:0.67, Val Loss: 0.36063
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.09400
E

Epoch:22
Validation Accuracy:77.48, Validation F1:0.77, Val Loss: 0.27310
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.06111
Epoch    22: reducing learning rate of group 0 to 1.0000e-08.
Epoch:23
Validation Accuracy:77.62, Validation F1:0.78, Val Loss: 0.27359
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.05997
Epoch:24
Validation Accuracy:77.14, Validation F1:0.77, Val Loss: 0.27632
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.06272
Epoch:25
Validation Accuracy:77.78, Validation F1:0.78, Val Loss: 0.26992
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.05773
Epoch:26
Validation Accuracy:77.21, Validation F1:0.77, Val Loss: 0.27644
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.06028
Epoch:27
Validation Accuracy:77.74, Validation F1:0.78, Val Loss: 0.27170
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.06172
Epoch:28
Validation Accuracy:77.04, Validation F1:0.77, Val Loss: 0.27785
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.06096


Epoch:32
Validation Accuracy:83.23, Validation F1:0.83, Val Loss: 0.21495
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02974
Epoch:33
Validation Accuracy:83.62, Validation F1:0.84, Val Loss: 0.21038
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.03363
Epoch:34
Validation Accuracy:83.40, Validation F1:0.83, Val Loss: 0.21029
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02884
Epoch:35
Validation Accuracy:83.09, Validation F1:0.83, Val Loss: 0.21363
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.03134
Epoch:36
Validation Accuracy:83.56, Validation F1:0.84, Val Loss: 0.21329
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.03008
Epoch:37
Validation Accuracy:83.17, Validation F1:0.83, Val Loss: 0.21276
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02981
Epoch:38
Validation Accuracy:83.42, Validation F1:0.83, Val Loss: 0.21230
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02899
Epoch:39
Validation Accuracy:83.33, Validation F1:0.83, Val Lo

Epoch:42
Validation Accuracy:89.60, Validation F1:0.90, Val Loss: 0.13957
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01390
Epoch:43
Validation Accuracy:89.65, Validation F1:0.90, Val Loss: 0.13826
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01379
Epoch:44
Validation Accuracy:89.50, Validation F1:0.90, Val Loss: 0.13927
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01423
Epoch:45
Validation Accuracy:89.53, Validation F1:0.90, Val Loss: 0.13967
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01437
Epoch:46
Validation Accuracy:89.55, Validation F1:0.90, Val Loss: 0.14232
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01445
Epoch:47
Validation Accuracy:89.69, Validation F1:0.90, Val Loss: 0.13577
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01406
Epoch:48
Validation Accuracy:89.50, Validation F1:0.90, Val Loss: 0.13825
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01427
Epoch:49
Validation Accuracy:89.52, Validation F1:0.90, Val Lo

In [12]:
mean = 0.
std = 0.
for images, _, _ in Train_loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images1 = images.reshape(batch_samples, images.size(1), -1)
    mean += images1.mean(2).sum(0)
    std += images1.std(2).sum(0)
    
mean /= len(Train_loader.dataset)
std /= len(Train_loader.dataset)

In [13]:
print(mean, std)

tensor([-24.3633]) tensor([14.2659])


In [65]:

model = conv_net().to(device)
model.load_state_dict(reqd_weights)

<All keys matched successfully>

In [9]:
pickle.dump(results_dict, open('Size_Exp_Jigsaw_Wt_Results.pkl'.format(sizes[i]),'wb'))

In [10]:
results_dict.keys()

dict_keys([10, 50, 250, 500])

In [8]:
import torchvision.models as models
resnet18 = models.resnet18().to(device)
alexnet = models.alexnet().to(device)
vgg16 = models.vgg16().to(device)
squeezenet = models.squeezenet1_0().to(device)
densenet = models.densenet161().to(device)
inception = models.inception_v3().to(device)


In [2]:
import pickle

results_dict = pickle.load(open('Size_Exp_Jigsaw_Wt_Results.pkl','rb'))

In [4]:
results_dict[1000]['val_acc_list']

[53.35987453251297,
 56.55688261551454,
 57.53408131258294,
 57.63059476414525,
 58.6681143684401,
 59.6453130655085,
 60.35709977078055,
 59.40402943660273,
 61.80480154421522,
 62.142598624683316,
 60.091687778984195,
 61.81686572566051,
 60.28471468210882,
 63.56617203522741,
 63.771263119797325,
 63.67474966823501,
 63.771263119797325,
 63.69887803112559,
 63.98841838581252,
 63.96429002292194,
 63.952225841476654,
 64.0125467487031,
 63.928097478586075,
 63.89190493425021,
 63.928097478586075,
 63.940161660031364,
 63.9039691156955,
 64.0004825672578,
 63.819519845578476,
 63.928097478586075,
 63.96429002292194,
 63.843648208469055,
 63.89190493425021,
 63.72300639401617,
 63.819519845578476,
 63.759198938352036,
 64.0125467487031,
 63.940161660031364,
 64.02461093014838,
 64.02461093014838,
 63.952225841476654,
 63.80745566413319,
 63.7953914826879,
 63.9039691156955,
 63.928097478586075,
 63.6868138496803,
 63.843648208469055,
 63.819519845578476,
 63.9039691156955,
 63.90396911